In [1]:
import requests
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup
import urllib

In [10]:
#The 'query' key contains the search terms exactly
url = 'http://api.elsevier.com/content/search/scopus?query=Apis+Mellifera&apiKey=582c2ee616e8c4119ae4201ce59a5e06&date=1980-2018&sort=+coverDate'
r = requests.get(url)
data = r.json()
results = int(data['search-results']['opensearch:totalResults'])
titles = set({""})

In [3]:
print(results)
login = open("./LoginInfo.txt","r")
user = login.readline()
password = login.readline()
login.close()

37753


In [4]:
abstractPage = requests.get("https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85039924208&origin=inward",auth = HTTPBasicAuth(user,password))
bSoupObj = BeautifulSoup(abstractPage.content,"html5lib")
abstractText = bSoupObj.find("section",{"id":"authorKeywords"}).findAll("span",{"class":"badges"})
# print abstractText
print(abstractText)

[<span class="badges">Hough transform</span>, <span class="badges">Kalman filter</span>, <span class="badges">Pollen detection</span>, <span class="badges">Pollen measurement</span>, <span class="badges">Receiver operating characteristic</span>]


In [5]:
sKeywords = ""
for word in bSoupObj.find("section",{"id":"indexedKeywords"}).findAll("span"):
    sKeywords+= word.text+", "
print(sKeywords[:-2])

Color, Food products, Food storage, Hough transforms, Elliptical shapes, Image moments, Object discrimination, Pollen counting, Pollen discriminations, Receiver operating characteristics, Tracking models, Video monitoring, Kalman filters


In [ ]:
f = open("../data/paperTable1.tsv","w")
entry_size = len(data['search-results']['entry'])
print entry_size

In [16]:
# f = open("../data/paperTable.tsv","w")
# entry_size = len(data['search-results']['entry'])
# start_point = 25
import threading
import CustomApi as api
dateRange = '1980-1996'
abstracts = []
def readAbstracts(start,stop):
    while(start < stop):
        entries = None
        try:
            url = 'http://api.elsevier.com/content/search/scopus?query=Apis+Mellifera&apiKey=582c2ee616e8c4119ae4201ce59a5e06&date=1980-2018&sort=+coverDate&start='+str(start)
            data = requests.get(url).json()
            entries = data['search-results']['entry']
        except:
            start += 25
            continue
        for entry in entries:
            #gets abstract text
            abstractPage = requests.get(entry['link'][2]['@href'],auth = HTTPBasicAuth(user,password))
            bSoupObj = BeautifulSoup(abstractPage.content,"html5lib")
            if(len(bSoupObj.findAll("p"))>1):
                abstractText = bSoupObj.findAll("p")[1].text
                #creates lists of keywords, seperated into author keywords and scopus assigned keywords
                aKeywords = ""
                sKeywords = ""
                if bSoupObj.find("section",{"id":"authorKeywords"}) is not None:
                    for word in bSoupObj.find("section",{"id":"authorKeywords"}).findAll("li"):
                        aKeywords+= word.text+", "
                if bSoupObj.find("section",{"id":"indexedKeywords"}) is not None:
                    for word in bSoupObj.find("section",{"id":"indexedKeywords"}).findAll("span"):
                        sKeywords+= word.text+", "

                #prints title, abstract, tags, and date created
                if abstractText.encode("utf8")=="[No abstract available]":
                    print "no abs"
                else:
                    try:
                        a = api.MyAbstract(title=entry['dc:title'].encode("utf8"),date=entry['prism:coverDate'].encode("utf8"),text=str(abstractText.encode("utf8")+"\t"+aKeywords[:-2].encode("utf8")+"\t"+sKeywords[:-2].encode("utf8")))
                        if a.title not in titles:
                            titles.add(a.title)
                            abstracts.append(a)
                        else:
                            pass
                        #f.write(entry['dc:title'].encode("utf8")+"\t"+abstractText.encode("utf8")+"\t"+aKeywords[:-2].encode("utf8")+"\t"+sKeywords[:-2].encode("utf8")+"\t"+entry['prism:coverDate'].encode("utf8")+"\n")
                    except Exception as ex:
                        print ex
                        pass
        start += 25
# thread1 = threading.Thread(target=readAbstracts,args=(0,1500))
# thread2 = threading.Thread(target=readAbstracts,args=(1500,2*dist))
# thread3 = threading.Thread(target=readAbstracts,args=(2*dist,3*dist))
# thread1.start()
# thread2.start()
# thread3.start()
# while True:
#     if thread1.isAlive() or thread2.isAlive() or thread3.isAlive():
#         continue
#     else:
#         break

readAbstracts(0,100)

no abs
no abs
no abs
no abs
no abs
no abs
no abs


In [ ]:
from datetime import datetime
f = lambda x : datetime.strptime(x.date,'%Y-%m-%d')
abstracts.sort(key=f)

In [ ]:
url = 'http://api.elsevier.com/content/search/scopus?query=Apis+Mellifera&apiKey=582c2ee616e8c4119ae4201ce59a5e06&date=1980-2018&sort=+coverDate&start='+str(0)
data = requests.get(url).json()
entries = data['search-results']['entry']
entries

In [15]:
len(abstracts)

92